In [3]:
#!pip install transformers
#!python -m spacy download en_core_web_lg

In [4]:
import os
import math
import gc

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import spacy
from nltk.stem.snowball import PorterStemmer

# Arquivos Pythons com funções e constantes
from qs_constants import AppConstants
from qs_functions import load_qa_pairs, normalize_qa, remove_duplication, remove_bad_answer, get_context_files
from qs_classes import TitleGroup, QuestionEmbedding

import tensorflow as tf

from transformers import AutoTokenizer, AutoModel

import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as t_funcional

from tqdm import tqdm

In [5]:
nlp = spacy.load('en_core_web_lg')
stemmer = PorterStemmer()

In [6]:
def carrega_dataset():
    df_all = load_qa_pairs()
    df_preparado = df_all[(df_all.Question.notna()) &
                          (df_all.Answer.notna())]
    vet_QA = df_preparado[["Question", "Answer", "ArticleTitle","ArticleFile"]].to_numpy()
    vet_QA = normalize_qa(vet_QA, lowercase=False)
    vet_QA = remove_bad_answer(vet_QA)
    vet_QA = remove_duplication(vet_QA)
    return vet_QA


In [7]:
def agrupa_perguntas_por_assunto(vet_QA):
    qa_agrupada_assunto = {}
    for qa in vet_QA:
        assunto = qa[AppConstants.COL_TITULO]
        assunto_preparado = assunto.lower().replace("_", " ")

        if assunto_preparado not in qa_agrupada_assunto:
            grupo = TitleGroup()
            qa_agrupada_assunto[assunto_preparado] = grupo
        else:
            grupo = qa_agrupada_assunto[assunto_preparado]

            
        q_embedding = QuestionEmbedding()
        q_embedding.question = qa[AppConstants.COL_PERGUNTA]
        q_embedding.answer = qa[AppConstants.COL_RESPOSTA]
        
        grupo.question_list.append(q_embedding)

        # existem arquivos com nan
        file = qa[AppConstants.COL_CONTEXT_FILE]
        if isinstance(file, str):
            grupo.files.add(file)
    return qa_agrupada_assunto
    
    

In [8]:
def carrega_contexto_assunto(qa_agrupada_assunto):
    with  tqdm(total=len(qa_agrupada_assunto), position=0, desc='Carregando Assuntos...') as progress:
        for k_grupo in qa_agrupada_assunto:    
            progress.set_description(f"Carregando Assunto [{k_grupo:<30}]")
            grupo = qa_agrupada_assunto[k_grupo]
            for ctx_file in grupo.files:
                file_path = os.path.join(AppConstants.CONTEXT_TEXT_PATH, ctx_file + AppConstants.CONTEXT_FILE_EXTENSION)
                with open(file_path, 'r', encoding='utf-8') as f_ctx:
                    ctx_file_doc = f_ctx.read()
                    ctx_nlp_doc = nlp(ctx_file_doc)
                    if ctx_nlp_doc.ents:
                        for ent in ctx_nlp_doc.ents:
                            entidade = ent.text
                            grupo.context_ner.add(entidade)
            
            progress.update(1)
        progress.set_description("Assuntos carregados.")

In [9]:
# retorna uma lista de asssuntos que tem potencial match com a pergunta
def get_assunto_pergunta(qa_agrupada_assunto, pergunta):
    doc_pergunta = nlp(pergunta)
    lista_assunto_match = []
    
    for k_assunto in qa_agrupada_assunto:
        grupo = qa_agrupada_assunto[k_assunto]
        achou = False
        como_achou = ''
        
        # NER pergunta
        if doc_pergunta.ents:
            for entidade in doc_pergunta.ents:
                entidade_text = entidade.text.lower()
                for entidade_token in entidade_text.split():
                    if entidade_token in k_assunto:
                        achou = True
                        como_achou = AppConstants.METODO_BUSCA_ASSUNTO_NER_PERGUNTA
                        
        # Prcura por nomes
        if not achou and doc_pergunta.noun_chunks:
            for noun in doc_pergunta.noun_chunks:
                for n_part in noun.text.split():
                    if n_part in k_assunto:
                        achou = True
                        como_achou = AppConstants.METODO_BUSCA_ASSUNTO_NOUN_PERGUNTA    
    
         # Lemmanization
        if not achou:
            doc_assunto = nlp(k_assunto)
            for token_assunto in doc_assunto:
                if token_assunto.is_stop:
                    continue
                assunto_lemma = token_assunto.lemma_.lower()
                for token_pergunta in doc_pergunta:
                    if token_pergunta.is_stop:
                        continue
                    if assunto_lemma == token_pergunta.lemma_.lower():
                        achou = True
                        como_achou = AppConstants.METODO_BUSCA_ASSUNTO_LEMMANIZATION
                if not achou:
                    for token_pergunta in doc_pergunta:
                        if token_pergunta.is_stop:
                            continue
                        if stemmer.stem(token_pergunta.text) == stemmer.stem(token_assunto.text):
                            achou = True
                            como_achou = AppConstants.METODO_BUSCA_ASSUNTO_STEMMER    
                    
        # NER context file            
        if not achou:
            if doc_pergunta.ents:
                for entidade in doc_pergunta.ents:
                    entidade_text = entidade.text
                    if entidade_text in grupo.context_ner:
                            achou = True
                            como_achou = AppConstants.METODO_BUSCA_ASSUNTO_NER_CONTEXTO
                            
        
        if achou:
            lista_assunto_match.append((k_assunto, como_achou))
            
    return lista_assunto_match

In [10]:
# dada uma lista de matchs para assunto retonra aquele que teoricamente seria o de maior peso, seguindo a ordem de prioridade por busca
def get_assunto_match_principal(lista_assunto_match):
    
    def encontra_metodo_busca(lista_assunto, metodo_busca_pesquisa):
        for (assunto, metodo_busca) in lista_assunto_match:
            if metodo_busca == metodo_busca_pesquisa:
                return assunto
        return None
    
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_NER_PERGUNTA)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_NER_PERGUNTA
        
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_NOUN_PERGUNTA)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_NOUN_PERGUNTA
    
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_LEMMANIZATION)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_LEMMANIZATION
    
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_STEMMER)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_STEMMER
    
    assunto_encontrado = encontra_metodo_busca(lista_assunto_match, AppConstants.METODO_BUSCA_ASSUNTO_NER_CONTEXTO)
    if assunto_encontrado:
        return assunto_encontrado, AppConstants.METODO_BUSCA_ASSUNTO_NER_CONTEXTO
    
    

In [11]:
# imprime as perguntas cuja a entidade não conseguiu ser encontrada
def teste_busca_contexto():
    vet_QA = carrega_dataset()
    qa_agrupada_assunto = agrupa_perguntas_por_assunto(vet_QA)
    carrega_contexto_assunto(qa_agrupada_assunto)

    total = 0
    qtd_achou_lista = 0
    qtd_achou_lista_errada = 0
    qtd_achou_unico = 0
    qtd_achou_unico_lista = 0
    qtd_nao_achou = 0
    qtd_achou_unico_incorreto = 0


    estatistica_busca = {AppConstants.METODO_BUSCA_ASSUNTO_NER_PERGUNTA: 0, 
                         AppConstants.METODO_BUSCA_ASSUNTO_NOUN_PERGUNTA : 0,
                         AppConstants.METODO_BUSCA_ASSUNTO_LEMMANIZATION : 0,
                         AppConstants.METODO_BUSCA_ASSUNTO_STEMMER : 0,
                         AppConstants.METODO_BUSCA_ASSUNTO_NER_CONTEXTO : 0,
                         'PERGUNTA_SEM_CONTEXTO' : 0
                        }

    for k_assunto in qa_agrupada_assunto:
        grupo = qa_agrupada_assunto[k_assunto]
        for qa in grupo.question_list:
            total += 1
            achou = False
            como_achou = ""

            pergunta = qa.question

            lista_assunto_match = get_assunto_pergunta(qa_agrupada_assunto, pergunta)

            if len(lista_assunto_match) == 1:
                (assunto_encontrado, metodo_busca) = lista_assunto_match[0]
                if assunto_encontrado != k_assunto:
                    qtd_achou_unico_incorreto += 1
                else:
                    qtd_achou_unico += 1
                    estatistica_busca[metodo_busca] += 1

            elif len(lista_assunto_match) > 0:
                assunto_encontrado, metodo_busca = get_assunto_match_principal(lista_assunto_match)
                if assunto_encontrado == k_assunto:
                    qtd_achou_unico_lista += 1
                    estatistica_busca[metodo_busca] += 1
                else:
                    achou_lista = False
                    for (assunto_encontrado, metodo_busca) in lista_assunto_match:
                        if assunto_encontrado != k_assunto:
                            continue
                        achou_lista = True
                    if achou_lista:
                        qtd_achou_lista += 1
                    else:
                        qtd_achou_lista_errada += 1
            else:
                print("Não achou ", k_assunto," - ", pergunta)
                qtd_nao_achou +=1
                for token_sem_ctx in AppConstants.TOKENS_COM_FALTA_CONTEXTO:
                    if token_sem_ctx in pergunta:
                        estatistica_busca['PERGUNTA_SEM_CONTEXTO'] += 1


    print ('Total:', total)
    print('Achou Unico', qtd_achou_unico)
    print('Achou Unico Pela Lista através de prioridade entre métodos ', qtd_achou_unico_lista)
    print('Achou Unico Incorreto', qtd_achou_unico_incorreto)
    print('Achou Lista ', qtd_achou_lista)
    print('Achou Lista Errada ', qtd_achou_lista_errada)
    print('Não Achou', qtd_nao_achou)
    print(estatistica_busca)

In [ ]:
teste_busca_contexto()

Assuntos carregados.: 100%|██████████| 105/105 [01:58<00:00,  1.13s/it]


Não achou  abraham lincoln  -  Did his mother die of pneumonia?


### Processando as Sentenças para geração de Embedding

In [13]:
# criação modelo, caminho Hugginface recebido por parametro
def create_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path)
    model.to('cuda')
    return model, tokenizer

In [14]:
# liberar espaço ocupado pelo modelo na GPU
def destroy_model(model):
    del model
    gc.collect()
    torch.cuda.empty_cache()

##### Prepara o dataset para entrar no modelo bert. Gera input_id, attention_mask e monta o data loaader para processamento em mini-batchs
- *Vetor de perguntas - Numpy*


In [15]:
def prepare_to_bert(np_sentences, tokenizer, sentence_max_length=450, batch_size=5):
    inputs = tokenizer.batch_encode_plus(np_sentences, 
                                         return_tensors='pt',
                                         padding='max_length',
                                         max_length=sentence_max_length)
    
    input_ids = inputs['input_ids']
    input_ids = input_ids.to('cuda')
    attention_mask = inputs['attention_mask']
    attention_mask = attention_mask.to('cuda')
    
    dataset = TensorDataset(input_ids, attention_mask)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    return loader   
    

##### Realiza o encode da sentença, retornando mean e mean polling*

In [16]:
def encode_sentence(sentences, bert_model, bert_tokenizer):
    sentence_loader = prepare_to_bert(sentences, bert_tokenizer, sentence_max_length=450, batch_size=5)

    mean_pooled_vet = []
    mean_vet = []
    
    for batch_id, (input_ids, attention_masks) in enumerate(sentence_loader):
        output = bert_model(input_ids, attention_mask=attention_masks)

        embeddings = output.last_hidden_state
        embeddings.detach()

        # calculo mean_pooling
        mask = attention_masks.unsqueeze(-1).expand(embeddings.size()).float()
        masked_embeddings = embeddings * mask
        summed = torch.sum(masked_embeddings, 1)
        summed_mask = torch.clamp(mask.sum(1), min=1e-9)
        mean_pooled = summed / summed_mask
        # convert from PyTorch tensor to numpy array
        mean_pooled = mean_pooled.cpu().data.numpy()
        mean_pooled_vet.append(mean_pooled)

        #calculo mean
        mean = embeddings.mean(dim=1)
        mean = mean.cpu().data.numpy()
        mean_vet.append(mean)

        #--- libera recursos da gpu
        del output
        gc.collect()
        torch.cuda.empty_cache()

    mean_pooled_vet = np.concatenate(mean_pooled_vet)
    mean_vet = np.concatenate(mean_vet)
    
    return mean_pooled_vet, mean_vet

In [17]:
# dada a base agrupada gera os embeddings por pergunta do assunto
def gera_embedding_pergunta_por_assunto(qa_agrupada_assunto):
    model, tokenizer = create_model(AppConstants.MODEL_BASE_NLI_MEAN)
    try:
        with  tqdm(total=len(qa_agrupada_assunto), position=0, desc='Encoding de Sentenças') as progress:
            for k_grupo in qa_agrupada_assunto:
                progress.set_description(f"Gerando Embedding Sentenças [{k_grupo:<30}]")
                grupo = qa_agrupada_assunto[k_grupo]
                vet_pergunta = []
                for q_embedding in grupo.question_list:
                    vet_pergunta.append(q_embedding.question)
                sentence_embeddings_mean_pooled, _ = encode_sentence(vet_pergunta, bert_model=model, bert_tokenizer=tokenizer)    
                for index, q_embedding in enumerate(grupo.question_list):
                    q_embedding.embedding = sentence_embeddings_mean_pooled[index]
                progress.update(1)
            progress.set_description("Geração Embedding Finalizada")
    finally:
        destroy_model(model)     
    

In [35]:
# dada uma lista de perguntas retorna os respectivos embeddings de sentença
# lista_pergunta - lista de perguntas que serão processadas
def gera_embedding_perguntas(lista_pergunta):
    model, tokenizer = create_model(AppConstants.MODEL_BASE_NLI_MEAN)
    try:
        sentence_embeddings_mean_pooled, _ = encode_sentence(lista_pergunta, bert_model=model, bert_tokenizer=tokenizer)                                    
    finally:
        destroy_model(model)    
    return sentence_embeddings_mean_pooled

In [38]:
# retorna as perguntas com maior similaridade 
# embedding_base_qa - lista de embeddings da base de perguntas e respostas
# embedding pergunta - embedding de uma pergunta
# return - lista de tuplas com (indice e similaridade)
def get_most_similarity_questions(embedding_base_qa, embedding_pergunta, threshold=0.7, max_question_return=3):
    similarities = t_funcional.cosine_similarity(torch.tensor([embedding_pergunta]), torch.tensor(embedding_base_qa))
    closest = similarities.argsort(descending=True)

    similarity_return = []
    qtd = 0
    for ind in closest:
        if similarities[ind] > threshold:
            similarity_return.append((ind, similarities[ind]))
        qtd += 1
        if qtd >= max_question_return:
            break

    return similarity_return
    

In [22]:
# Retorna um array de embeddings dada uma lista de perguntas
def question_to_embedding_array(question_list):
    resp = []
    for question in question_list:
        resp.append(question.embedding)
    return resp

## Execução

In [23]:
vet_QA = carrega_dataset()
qa_agrupada_assunto = agrupa_perguntas_por_assunto(vet_QA)
carrega_contexto_assunto(qa_agrupada_assunto)
gera_embedding_pergunta_por_assunto(qa_agrupada_assunto)
    


Assuntos carregados.: 100%|██████████| 105/105 [01:49<00:00,  1.04s/it]


Some weights of the model checkpoint at sentence-transformers/bert-base-nli-mean-tokens were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Geração Embedding Finalizada: 100%|██████████| 105/105 [02:37<00:00,  1.50s/it]


In [ ]:
print(qa_agrupada_assunto['abraham lincoln'].question_list[0].question)
print(qa_agrupada_assunto['abraham lincoln'].question_list[0].embedding)

In [44]:
MIN_SIMILARIY = 0.7
question_list = ["Was Abraham Lincoln the president of the United States?",
                 "Why ducks don't fly?",
                 "Can use ducks for economics use?",
                 "Is ducks confused with other animals?",
                 "How many years elephants live?",
                 "How long for elephant's get a baby?",
                 "Did Finland have an international crisis with another country?",
                 "Can I use ants for cooking?",
                 "Does ants fly?",
                 "Can i hear the butterflies?",
                 "Was Anders Celsius born in Sweden?",
                 "What you can tell me about cello?"]

embedding_perguntas =  gera_embedding_perguntas(question_list)

for ind_pergunta, pergunta in enumerate(question_list):
    print('-----------------------------------------------------------------------------------------------')
    print('Pergunta:', pergunta)
    lista_assuntos = get_assunto_pergunta(qa_agrupada_assunto, pergunta)
    lista_questions_to_compare = []
    if (len(lista_assuntos) > 0):
        for k_assunto, _ in lista_assuntos:
            grupo = qa_agrupada_assunto[k_assunto]
            for question in grupo.question_list:
                lista_questions_to_compare.append(question)
        embedding_base_question_array = question_to_embedding_array(lista_questions_to_compare)
        questions_similarities = get_most_similarity_questions(embedding_base_question_array, 
                                                               embedding_perguntas[ind_pergunta], 
                                                               threshold=MIN_SIMILARIY)        
        if (len(questions_similarities) == 0):
            print("Nenhuma pergunta encontrada com a similaridade maior que o mínimo estipulado [", MIN_SIMILARIY , "]")
            continue

        print(len(questions_similarities), ' respostas encontradas com similaridade maior que ' , MIN_SIMILARIY) 
        for ind_sim, cos_sim in questions_similarities:
            print(lista_questions_to_compare[ind_sim].question, ' - ', cos_sim )

    else:
        print("Nenhum assunto encontrado")
    print('-----------------------------------------------------------------------------------------------')

Some weights of the model checkpoint at sentence-transformers/bert-base-nli-mean-tokens were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


-----------------------------------------------------------------------------------------------
Pergunta: Was Abraham Lincoln the president of the United States?
3  respostas encontradas com similaridade maior que  0.7
Was Abraham Lincoln the first President of the United States?  -  tensor(0.9469)
When did Lincoln first serve as President?  -  tensor(0.8925)
Was James Monroe President of the United States?  -  tensor(0.8230)
-----------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------
Pergunta: Why ducks don't fly?
3  respostas encontradas com similaridade maior que  0.7
What unrelated water birds are ducks sometimes confused with?  -  tensor(0.7844)
What makes it more difficult for a diving duck to fly?  -  tensor(0.7573)
What types of unrelated water birds with similar forms are ducks sometimes confused with?  -  tensor(0.7022)
--------------------------